# lasagne
* lasagne is a library for neural network building and training
* it's a low-level library with almost seamless integration with theano

For a demo we shall solve the same digit recognition problem, but at a different scale
* images are now 28x28
* 10 different digits
* 50k samples

In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from mnist import load_dataset
X_train,y_train,X_val,y_val,X_test,y_test = load_dataset()

print(X_train.shape,y_train.shape)

(50000, 1, 28, 28) (50000,)


In [3]:
input_X = T.tensor4("X")

#input dimention (None means "Arbitrary" and only works at the first axes [samples])
input_shape = [None,1,28,28] # (batchsize, channels, rows, columns)

target_y = T.vector("target Y integer", dtype='int32')

Defining network architecture

In [29]:
import lasagne
from lasagne.layers import *

# Input layer (auxilary)
l1 = InputLayer(shape = input_shape, input_var=input_X)

# fully connected layer, that takes input layer and applies 50 neurons to it.
# nonlinearity here is sigmoid as in logistic regression
# you can give a name to each layer (optional)

# Convolution + Pooling
# Convolutional Layer receives input from the layer l1, and has 6 filters of size 5x5
l2_conv = Conv2DLayer(l1, num_filters=10, filter_size=5, nonlinearity=lasagne.nonlinearities.linear)
l2_pool = Pool2DLayer(l2_conv, pool_size=2)

# nonlinearities:
# elu, sigmoid, tanh, ScaledTanH, softmax, rectify, LeakyRectify, 
# leaky_rectify, very_leaky_rectify, softplus, linear, identity

l3_conv = Conv2DLayer(l2_pool, num_filters=20, stride = 1, filter_size=5, nonlinearity=lasagne.nonlinearities.leaky_rectify)
l3_pool = Pool2DLayer(l3_conv, pool_size=2)

l4 = DenseLayer(l3_pool, num_units=200, nonlinearity = lasagne.nonlinearities.sigmoid)
l4_drop = DropoutLayer(l4, p=0.5)
l5 = DenseLayer(l4_drop, num_units=400, nonlinearity = lasagne.nonlinearities.tanh)

# fully connected output layer that takes dense_1 as input and has 10 neurons (1 for each digit)
# We use softmax nonlinearity to make probabilities add up to 1
l_out = DenseLayer(l5, num_units = 10, nonlinearity=lasagne.nonlinearities.softmax)


In [30]:
#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(l_out)

In [31]:
#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(l_out)
print(all_weights)

[W, b, W, b, W, b, W, b, W, b]


### Then you could simply
* define loss function manually
* compute error gradient over all weights
* define updates
* But that's a whole lot of work and life's short
  * not to mention life's too short to wait for SGD to converge

Instead, we shall use Lasagne builtins

In [32]:
# Define hyperparameters. These could also be symbolic variables
lr = 1e-2
weight_decay = 1e-5

#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = lasagne.objectives.categorical_crossentropy(y_predicted,target_y).mean()

# Also add weight decay to the cost function
weightsl2 = lasagne.regularization.regularize_network_params(l_out, lasagne.regularization.l2)
loss += weight_decay * weightsl2

#prediction accuracy
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.sgd(loss, all_weights,learning_rate=0.1)

In [33]:
#function that computes loss and updates weights
train_fun = theano.function([input_X,target_y],[loss,accuracy],updates= updates_sgd)

#function that just computes accuracy
accuracy_fun = theano.function([input_X,target_y],accuracy)

### That's all, now let's train it!
* We got a lot of data, so it's recommended that you use SGD
* So let's implement a function that splits the training sample into minibatches

In [34]:
# An auxilary function that returns mini-batches for neural network training

# Parameters
# inputs - a tensor of images with shape (many, 1, 28, 28), e.g. X_train
# outputs - a vector of answers for corresponding images e.g. Y_train
# batch_size - a single number - the intended size of each batches

def iterate_minibatches(inputs, targets, batchsize):
    assert len(inputs) == len(targets)
    indices = np.arange(len(inputs))
    np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        excerpt = indices[start_idx:start_idx + batchsize]
        yield inputs[excerpt], targets[excerpt]

# Training loop

In [35]:
import time

num_epochs = 150 # amount of passes through the data

batch_size = 40 # number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 150 took 90.315s
  training loss (in-iteration):		0.407941
  train accuracy:		86.73 %
  validation accuracy:		95.82 %
Epoch 2 of 150 took 118.153s
  training loss (in-iteration):		0.127597
  train accuracy:		96.25 %
  validation accuracy:		97.08 %
Epoch 3 of 150 took 122.890s
  training loss (in-iteration):		0.096045
  train accuracy:		97.22 %
  validation accuracy:		97.71 %
Epoch 4 of 150 took 120.749s
  training loss (in-iteration):		0.080067
  train accuracy:		97.69 %
  validation accuracy:		97.96 %
Epoch 5 of 150 took 120.011s
  training loss (in-iteration):		0.070435
  train accuracy:		98.02 %
  validation accuracy:		98.10 %
Epoch 6 of 150 took 120.105s
  training loss (in-iteration):		0.065196
  train accuracy:		98.14 %
  validation accuracy:		98.19 %
Epoch 7 of 150 took 128.925s
  training loss (in-iteration):		0.058180
  train accuracy:		98.44 %
  validation accuracy:		98.20 %
Epoch 8 of 150 took 120.358s
  training loss (in-iteration):		0.052796
  train accuracy:		9

In [36]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 99:
    print ("Achievement unlocked: 80lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		99.02 %
Achievement unlocked: 80lvl Warlock!


# Now improve it!

* Moar layers!
* Moar units!
* Different nonlinearities!